In [1]:
import pandas as pd
import numpy as np

In [38]:
models = {}

In [58]:
df = pd.read_csv('3_processed.csv')


In [59]:
df.head()

,Unnamed: 0,text,question_mark_check,hashtag_check,url_check,lexical_diversity,words_count,sentiment,trend_score,image,image_type,video,engagement_score
0,0,1,0,0,0,1.000000,18,1.000000,1.00,0,0,0,0.000187
1,1,1,0,0,1,0.956522,23,0.400000,1.00,0,0,0,0.000030
2,9,1,0,0,0,1.000000,9,0.000000,14.35,0,0,0,0.000365
3,10,1,0,0,1,0.933333,15,0.386667,0.00,0,0,0,0.000039
4,14,1,0,0,0,0.900000,10,0.000000,25.15,0,0,0,0.000197


In [60]:
df.columns

Index(['Unnamed: 0', 'text', 'question_mark_check', 'hashtag_check',
       'url_check', 'lexical_diversity', 'words_count', 'sentiment',
       'trend_score', 'image', 'image_type', 'video', 'engagement_score'],
      dtype='object')

In [61]:
X = df.iloc[:, 1:12].values
y = df.iloc[:, 12].values

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [63]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [64]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=12)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [65]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.0002146731473654732
Mean Squared Error: 1.004848106364566e-07
Root Mean Squared Error: 0.00031699339210219604


In [66]:
models['pypcom'] = [sc,regressor]

In [70]:
models

{'143322019583033': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)],
 '546588969267691': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)],
 'InsaneTech': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)],
 'pypcom': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)]}

In [71]:
import pickle

In [72]:
with open('models.pickle', 'wb') as handle:
    pickle.dump(models, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [73]:
import re
from textblob import TextBlob
from meme_or_not import set_model_for_image_type,get_image_type
from pytrends.request import TrendReq

with open('models.pickle', 'rb') as handle:
    rf_models = pickle.load(handle)

pytrends = TrendReq(hl='en-US', tz=530, timeout=(10,25), retries=3, backoff_factor=0.2)
model = set_model_for_image_type()

In [74]:
rf_models

{'143322019583033': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)],
 '546588969267691': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)],
 'InsaneTech': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)],
 'pypcom': [StandardScaler(),
  RandomForestRegressor(n_estimators=20, random_state=12)]}

In [11]:
def get_metrics(post,image = False,video = False):

    def question_mark_check(post):
        #print(row['text'])
        if '? ' not in str(post):
            val = 0
        else:
            val = 1
        return val

    def hashtag_check(post):
        #print(row['text'])
        if '#' not in str(post):
            val = 0
        else:
            val = str(post).count('#')
        return val

    def url_check(post):

        regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        url = re.findall(regex,str(post))
        if len(url)>0:
            return 1
        else:
            return 0

    def words_count(post):
        return len(str(post).split())

    def lexical_diversity(post):
        return len(set(str(post).split())) / len(str(post).split())


    def sentiment(post):
        testimonial = TextBlob(str(post))
        return testimonial.sentiment.polarity

    def image_check(image):
        if not image:
            return 0
        else:
            x = get_image_type(model,image)
            return x
  
    def trend_status(post):

        from rake_nltk import Rake

        r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.

        r.extract_keywords_from_text(str(post))

        words = r.get_ranked_phrases()

        proper_nouns = set()
        tb = TextBlob(str(post))
        for i in tb.tags:
            if i[1] == 'NNP':
                proper_nouns.add(str(i[0]))

        proper_nouns = sorted(list(proper_nouns),key = len, reverse = True)[:5]



        if len(words) == 0:
            return 0,[]
        kw_list = sorted(list(words),key = len, reverse = True)[:5] + proper_nouns
        #print(kw_list)
        try:
            pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='IN', gprop='')
            interest = pytrends.interest_over_time().tail(20)

            interest = interest[interest.columns[0]]
        except:
            if len(proper_nouns) < 1:
                return 0,kw_list
            #print(proper_nouns)
            pytrends.build_payload(proper_nouns, cat=0, timeframe='today 5-y', geo='IN', gprop='')
            interest = pytrends.interest_over_time().tail(20)

            try:
                interest = interest[interest.columns[0]]
            except:
                return 0,kw_list

        x = interest.mean()
        if x == 0:
            #print("no interest")
            if len(proper_nouns) < 1:
                return 0,kw_list
            #print(proper_nouns)
            pytrends.build_payload(proper_nouns, cat=0, timeframe='today 5-y', geo='IN', gprop='')
            interest = pytrends.interest_over_time().tail(20)

            try:
                interest = interest[interest.columns[0]]
            except:
                #print('fail')
                return 0,kw_list
            return interest.mean(),kw_list
        return x,kw_list


    def text_boolean(post):
        #print(row[''])
        if len(str(post)) > 0:
            val = 1
        else:
            val = 0
        return val
    
    post = input()
    trend_s, keywords = trend_status(post)
    if not image:
        image_status = 0
        image_type = 0
    else:
        image_status = 1
        image_type = image_check(image)
    if not video:
        video_status = 0
    else:
        video_status = 1
    parameters = [1,question_mark_check(post),hashtag_check(post),url_check(post),lexical_diversity(post),
                 words_count(post), sentiment(post),trend_s,image_status,image_type,video_status]
    #parameters.append(int(input("image?")))
    #parameters.append(int(input("image type?")))
    #parameters.append(int(input("video?")))
    #parameters.append(input("engagement_score ?"))
    
    return parameters,keywords

In [75]:
def get_suggestions(post,group_id,image = False,video = False):
    
    x,kw = get_metrics(post,image,video)
    second = x.copy()
    second[1] = 1 #If no question add question

    third = x.copy()
    third[2] += 1 #Increase hashtag by 1

    fourth = x.copy()
    fourth[2] += 3#Increase hashtag by 3

    fifth = x.copy()
    fifth[3] = 1#If no url, add

    sixth = x.copy()
    sixth[5] += int(0.25*sixth[5]) #Increase word count by 25%

    seventh = x.copy()
    seventh[5] += int(0.5*seventh[5])#Increase word count by 50%


    eight = x.copy()
    eight[5] += int(0.75*eight[5])#Increase word count by 75% 

    ninth = x.copy()
    ninth[5] -= int(0.25*ninth[5])#Decrease word count by 25%

    tenth = x.copy()
    tenth[5] -= int(0.5*tenth[5])#Decrease word count by 50%

    eleventh = x.copy()
    eleventh[5] -= int(0.75*eleventh[5])#Decrease word count by 75%

    twelfth = x.copy()
    twelfth[5] = 5#Word count as 5

    thirteen = x.copy()
    thirteen[6] += 0.1*thirteen[6]#Increase sentiment by 10%

    fourteen = x.copy()
    fourteen[6] += 0.25*fourteen[6]#Increase sentiment by 25%

    fifteen = x.copy()
    fifteen[7] += 0.1*fifteen[7]#Increase trend score by 10%

    sixteen = x.copy()
    sixteen[7] += 0.25*sixteen[7]#Increase trend score by 25%

    seventeen = x.copy()
    seventeen[8],seventeen[9] = 1,1#Add image as meme

    eighteen = x.copy()
    eighteen[8],eighteen[9] = 1,-1#Add image as infogrphic

    nineteen = x.copy()
    nineteen[8],nineteen[9],nineteen[10] = 0,0,1#Add video


    to_predict = [x,
                 second,third,fourth,fifth,sixth,seventh,eight,ninth,tenth,eleventh,twelfth,thirteen,
                 fourteen,fifteen,sixteen,seventeen,eighteen,nineteen]
    
    sc = rf_models[group_id][0]
    regressor = rf_models[group_id][1] 
    input_ex = sc.transform(to_predict)
    y_pred = regressor.predict(input_ex)
    
    best_suggestion = np.argmax(y_pred) + 2
    
    return define_best_suggestion(best_suggestion)
    

In [76]:
def define_best_suggestion(p):
    
    if p == 2:
        return 'Adding a question to your post increases your posts engagement score'
    elif p == 3:
        return 'Try adding a hashtag. It seems like your post might do better'
    elif p == 3:
        return 'Try adding a hashtag. It seems like your post might do better'
    elif p == 4:
        return 'Adding 3 hashtags more seems like it will make a significant difference. Your post might do better'
    elif p == 5:
        return 'Can your post include a link to an article/video? Try adding this for better engagement'
    elif p == 6:
        return 'Number of words make a difference. A 25% increase in number of words in your post seems promising'
    elif p == 7:
        return 'Number of words make a difference. A 50% increase in number of words in your post seems promising'
    elif p == 8:
        return 'Number of words make a difference and it seems your post is too small. A 75% increase in number of words in your post seems promising'
    elif p == 9:
        return 'Number of words make a difference. A 25% decrease in number of words in your post seems promising'
    elif p == 10:
        return 'Your post is a bit too lengthy. If you try reducing the length of the post to about 50% its length, it may do much better'
    elif p == 11:
        return 'Your post is a bit too lengthy. If you try reducing the length of the post to about 25% its length, it may do much better'
    elif p == 12:
        return 'Try keeping the number of words limited to around 5. It look like for this post something short seems perfect'
    elif p == 13:
        return 'If the positive sentiment conveyed in your tweet is increased by 10%, it will do better'
    elif p == 14:
        return 'If the positive sentiment conveyed in your tweet is increased by 25%, it will do better'
    elif p == 15:
        return 'Including topics in trend in your tweet is always a good thing. Try including a few buzzwords. A 10% increase in trend score will help'
    elif p == 16:
        return 'Including topics in trend in your tweet is always a good thing. Try including a few buzzwords. A 25% increase in trend score will help'
    elif p == 17:
        return 'Including images in your post is always a good idea. Adding a meme or a image conveying humour might help'
    elif p == 18:
        return 'Including images in your post is always a good idea. Adding an image of a graph or some sort of infographic seems perfect'
    elif p == 19:
        return 'Including videos in your post is helpful at times. Posting a video seems like it helps engagement score'

In [22]:
#original
second = x.copy()
second[1] = 1 #If no question add question

third = x.copy()
third[2] += 1 #Increase hashtag by 1

fourth = x.copy()
fourth[2] += 3#Increase hashtag by 3

fifth = x.copy()
fifth[3] = 1#If no url, add

sixth = x.copy()
sixth[5] += int(0.25*sixth[5]) #Increase word count by 25%

seventh = x.copy()
seventh[5] += int(0.5*seventh[5])#Increase word count by 50%


eight = x.copy()
eight[5] += int(0.75*eight[5])#Increase word count by 75% 

ninth = x.copy()
ninth[5] -= int(0.25*ninth[5])#Decrease word count by 25%

tenth = x.copy()
tenth[5] -= int(0.5*tenth[5])#Decrease word count by 50%

eleventh = x.copy()
eleventh[5] -= int(0.75*eleventh[5])#Decrease word count by 75%

twelfth = x.copy()
twelfth[5] = 5#Word count as 5

thirteen = x.copy()
thirteen[6] += 0.1*thirteen[6]#Increase sentiment by 10%

fourteen = x.copy()
fourteen[6] += 0.25*fourteen[6]#Increase sentiment by 25%

fifteen = x.copy()
fifteen[7] += 0.1*fifteen[7]#Increase trend score by 10%

sixteen = x.copy()
sixteen[7] += 0.25*sixteen[7]#Increase trend score by 25%

seventeen = x.copy()
seventeen[8],seventeen[9] = 1,1#Add image as meme

eighteen = x.copy()
eighteen[8],eighteen[9] = 1,-1#Add image as infogrphic

nineteen = x.copy()
nineteen[8],nineteen[9],nineteen[10] = 0,0,1#Add video


to_predict = [x,
             second,third,fourth,fifth,sixth,seventh,eight,ninth,tenth,eleventh,twelfth,thirteen,
             fourteen,fifteen,sixteen,seventeen,eighteen,nineteen]

In [23]:
to_predict

[[1, 0, 4, 1, 0.8169014084507042, 71, 0.8, 57.3, 1, -1, 0],
 [1, 1, 4, 1, 0.8169014084507042, 71, 0.8, 57.3, 1, -1, 0],
 [1, 0, 5, 1, 0.8169014084507042, 71, 0.8, 57.3, 1, -1, 0],
 [1, 0, 7, 1, 0.8169014084507042, 71, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 71, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 88, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 106, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 124, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 54, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 36, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 18, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 5, 0.8, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 71, 0.8800000000000001, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 71, 1.0, 57.3, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 71, 0.8, 63.03, 1, -1, 0],
 [1, 0, 4, 1, 0.8169014084507042, 71, 0.8, 71.625, 1, -1, 0],
 [1, 0, 4, 1, 0.81690

In [24]:
input_ex = sc.transform(to_predict)

In [25]:
y_pred = regressor.predict(input_ex)

In [26]:
y_pred

array([0.000214  , 0.00021598, 0.000214  , 0.000214  , 0.000214  ,
       0.000214  , 0.000214  , 0.000214  , 0.000214  , 0.00021598,
       0.00024211, 0.00015705, 0.000214  , 0.000214  , 0.000214  ,
       0.000214  , 0.000214  , 0.000214  , 0.00021647])

In [27]:
best_suggestion = np.argmax(y_pred) + 2

In [28]:
best_suggestion

12